## Data Pipline 

Preparing individual dataset for merging  

Train and transactions obtained from eda.ipynb 
Holidays, Oil , Items , and stores obtained from Data_Overview

In [1]:
import pandas as pd
import seaborn as sns 
import numpy as np 
import datetime
import os 
from matplotlib import pyplot as plt
import random
import dask.dataframe as dd
from sklearn import preprocessing

Train

In [38]:
# optimizing size of data types 
dtype_dict={"id":'int32',
            "store_nbr":'int8',
            "item_nbr":'int32',
            "unit_sales":'float32',
            
           }
df1 = pd.read_csv("/Volumes/HD David/COSC490/favorita-grocery-sales-forecasting/train.csv",dtype=dtype_dict,parse_dates=['date'])

/Users/davidpinosproano/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [74]:

top10 = df1.item_nbr.value_counts().head(10).index


In [78]:
df1 = df1[df1.item_nbr.isin(top10)].reset_index(drop = True)

In [79]:
df1['onpromotion'] = df1['onpromotion'].fillna(0)
df1['onpromotion'] = df1['onpromotion'].astype('int8')
df1.dtypes



id                      int32
date           datetime64[ns]
store_nbr                int8
item_nbr                int32
unit_sales            float32
onpromotion              int8
dtype: object

Transactions 

In [80]:
dtype_dict={"id":'int32',
            "store_nbr":'int8',
            "transactions":'int32',

           }
transactions= pd.read_csv("/Volumes/HD David/COSC490/favorita-grocery-sales-forecasting/transactions.csv",dtype=dtype_dict,parse_dates=['date'])


In [81]:
transactions.dtypes


date            datetime64[ns]
store_nbr                 int8
transactions             int32
dtype: object

Holidays

In [82]:

holidays = pd.read_csv("/Volumes/HD David/COSC490/Clean_Data/holidays_clean.csv", parse_dates = ['date'])
holidays = holidays.drop(columns = ['Unnamed: 0'])

In [83]:
# Cleaning holidays table to generate a df with columns date and holiday(bool)

holidays = holidays.drop(holidays[holidays.transferred== True ].index)
holidays = holidays[(holidays.type == 'Holiday') | (holidays.type == 'Bridge')]
holidays = holidays[holidays.locale == 'National']
holidays['holiday'] = 1


In [84]:
#list of national holidays 
holidays = holidays[['date','holiday']]

Oil 

In [85]:
oil = pd.read_csv("/Volumes/HD David/COSC490/Clean_Data/oil_clean.csv", parse_dates = ['date'])
oil = oil.drop(columns = ['Unnamed: 0'])

Items 

In [86]:
items = pd.read_csv("/Volumes/HD David/COSC490/Clean_Data/items_clean.csv")
items = items.drop(columns = ['Unnamed: 0'])

Stores

In [87]:
stores = pd.read_csv("/Volumes/HD David/COSC490/Clean_Data/stores_clean.csv")
stores = stores.drop(columns = ['Unnamed: 0'])

# City only 


In [88]:

lis = stores[stores['city']=="Quito"]['store_nbr'].tolist()


# Sample from the Data 

In [90]:
# sampling 10% of the training dataset
random.seed(10000)

sample1 = df1.sample(n= round(len(df1)*0.5),random_state = 100)


# Join Tables

In [91]:
sample1

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,84,2013-01-01,25,261052,1.0,0
1,92,2013-01-01,25,265559,2.0,0
2,98,2013-01-01,25,273528,2.0,0
3,116,2013-01-01,25,314384,8.0,0
4,151,2013-01-01,25,364606,5.0,0
...,...,...,...,...,...,...
825645,125495909,2017-08-15,54,414353,12.0,0
825646,125495958,2017-08-15,54,502331,59.0,0
825647,125495995,2017-08-15,54,559870,6.0,0
825648,125496000,2017-08-15,54,564533,15.0,0


In [92]:
# worked well on merging trasnactions with 10 percent of the data 
train_tran = sample1.merge(transactions,on = ['date','store_nbr'],how = 'inner')

In [93]:
train_tran_stores = train_tran.merge(stores,on = "store_nbr",how = "inner")

In [94]:
train_tran_stores_oil = train_tran_stores.merge(oil, on = "date",how = "inner")

In [95]:
train_tran_stores_oil_items = train_tran_stores_oil.merge(items,on = "item_nbr", how = "inner")

In [96]:
#holiday yes no has to be binary 

# holidays[['date','locale']].sort_values(by = 'date')
train_tran_stores_oil_items_hol = train_tran_stores_oil_items.merge(holidays, on = 'date',how = 'left')


# Exploration
-----------------------

In [97]:
# verify joins working as needed
print("sample")
print(len(sample1))
print("transactions")
print(len(train_tran))
print("sotores")
print( len(train_tran_stores))
print("oil")
print( len(train_tran_stores_oil))
print("items")
print(len(train_tran_stores_oil_items))
print("hol")
print( len(train_tran_stores_oil_items_hol))


sample
825650
transactions
824494
sotores
824494
oil
824494
items
824494
hol
824494


In [98]:
# doing a left join produces 1 percent more data due to more than one holiday landing on the same day. 
12528238/12549704

0.9982895214102261

Final feature engineering 

In [99]:
train_tran_stores_oil_items_hol['holiday'] = train_tran_stores_oil_items_hol['holiday'].fillna(0)

In [100]:
# this is the output after removing cluster,type,city,state,and class using a sample of 10 percent of the data. 

final_sample = train_tran_stores_oil_items_hol.drop(columns = ['cluster','type','city','state','class'])


In [101]:
# make categorical variables into numerical
# using sklearn encoding
# we need to categorize  store number , item _nbr 
final_sample['item_nbr'].value_counts()

502331     83359
314384     83332
364606     83192
265559     82931
559870     82397
1036689    82020
273528     81999
564533     81968
261052     81656
414353     81640
Name: item_nbr, dtype: int64

In [102]:
final_sample

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,oil_price,family,perishable,holiday
0,84,2013-01-01,25,261052,1.0,0,770,93.14,GROCERY I,0,1.0
1,19934,2013-01-02,25,261052,3.0,0,1038,93.14,GROCERY I,0,0.0
2,737,2013-01-02,1,261052,23.0,0,2111,93.14,GROCERY I,0,0.0
3,1762,2013-01-02,2,261052,38.0,0,2358,93.14,GROCERY I,0,0.0
4,2875,2013-01-02,3,261052,92.0,0,3487,93.14,GROCERY I,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
824489,94998903,2016-10-26,20,1036689,8.0,0,1220,48.75,GROCERY I,0,0.0
824490,95011970,2016-10-26,29,1036689,9.0,0,945,48.75,GROCERY I,0,0.0
824491,95000630,2016-10-26,21,1036689,14.0,0,872,48.75,GROCERY I,0,0.0
824492,95032110,2016-10-26,42,1036689,29.0,0,1017,48.75,GROCERY I,0,0.0


In [103]:
# scikit learn exploration 
from sklearn.preprocessing import LabelEncoder


In [104]:
# encoding categorical variable 
final_sample['family_cat'] = LabelEncoder().fit_transform(final_sample['family'])

In [105]:
final_sample

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,transactions,oil_price,family,perishable,holiday,family_cat
0,84,2013-01-01,25,261052,1.0,0,770,93.14,GROCERY I,0,1.0,1
1,19934,2013-01-02,25,261052,3.0,0,1038,93.14,GROCERY I,0,0.0,1
2,737,2013-01-02,1,261052,23.0,0,2111,93.14,GROCERY I,0,0.0,1
3,1762,2013-01-02,2,261052,38.0,0,2358,93.14,GROCERY I,0,0.0,1
4,2875,2013-01-02,3,261052,92.0,0,3487,93.14,GROCERY I,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
824489,94998903,2016-10-26,20,1036689,8.0,0,1220,48.75,GROCERY I,0,0.0,1
824490,95011970,2016-10-26,29,1036689,9.0,0,945,48.75,GROCERY I,0,0.0,1
824491,95000630,2016-10-26,21,1036689,14.0,0,872,48.75,GROCERY I,0,0.0,1
824492,95032110,2016-10-26,42,1036689,29.0,0,1017,48.75,GROCERY I,0,0.0,1


In [106]:
# final_sample.to_csv("/Volumes/HD David/COSC490/Clean_Data/final_sample_top10.csv")
